In [1]:
import pandas as pd

### Analysing the output of:

`steadiercom *.xml -c abundance.tsv -m MILK --mediadb milk_composition.tsv --growth 0.05 --unlimited unlimited.txt --sample 100` 

- growth rate grossly estimated from figure 1 (assuming 25-fold total biomass increase in 90 hours)
- relative abundances also grossly taken from figure 1 (500:250:200:50)

In [7]:
df = pd.read_csv('output.tsv', sep='\t')

In [3]:
df.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
88,L_kefiranofaciens,L_mesenteroides,M_arg__L_e,0.000506,0.002886,0.91,kefir,MILK


In [4]:
df.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and frequency > 0.1' )

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
27,L_mesenteroides,L_kefiranofaciens,M_acald_e,0.004578,0.103917,0.44,kefir,MILK
123,L_mesenteroides,L_kefiranofaciens,M_h2s_e,0.000228,0.006697,0.44,kefir,MILK
146,L_mesenteroides,L_kefiranofaciens,M_nh4_e,0.000110,0.006077,0.76,kefir,MILK


In [5]:
df.query('donor == "L_lactis" and receiver == "A_fabarum" and frequency > 0.1')

,donor,receiver,compound,mass_rate,rate,frequency,community,medium
29,L_lactis,A_fabarum,M_4abut_e,0.004017,0.038957,0.23,kefir,MILK
90,L_lactis,A_fabarum,M_glyc_e,0.000496,0.005391,1.00,kefir,MILK


In [6]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

selected = df.query("frequency > 0.1 and mass_rate > 0.001 and donor != 'environment' and receiver != 'environment'")

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['mass_rate'])
    net.add_edge(row['compound'], row['receiver'], value=row['mass_rate'])

net.show('network.html')

tmp1.html


### SMETANA

In [2]:
df2 = pd.read_csv('detailed.tsv', sep='\t')

In [3]:
df2.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and smetana > 0.1')

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
35,kefir,MILK,L_mesenteroides,L_kefiranofaciens,M_acald_e,NaN,0.17,1,0.17
38,kefir,MILK,L_mesenteroides,L_kefiranofaciens,M_glc__D_e,NaN,0.45,1,0.45
39,kefir,MILK,L_mesenteroides,L_kefiranofaciens,M_h2s_e,NaN,0.82,1,0.82


In [5]:
df2.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and smetana > 0.1' )

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
8,kefir,MILK,L_kefiranofaciens,L_mesenteroides,M_acald_e,NaN,0.29,1,0.29
9,kefir,MILK,L_kefiranofaciens,L_mesenteroides,M_h2s_e,NaN,0.88,1,0.88


In [6]:
df2.query('donor == "L_lactis" and receiver == "A_fabarum" and smetana > 0.1')

,community,medium,receiver,donor,compound,scs,mus,mps,smetana
61,kefir,MILK,A_fabarum,L_lactis,M_4abut_e,NaN,0.18,1,0.18
62,kefir,MILK,A_fabarum,L_lactis,M_acald_e,NaN,0.18,1,0.18
64,kefir,MILK,A_fabarum,L_lactis,M_glyc_e,NaN,0.18,1,0.18
65,kefir,MILK,A_fabarum,L_lactis,M_h2s_e,NaN,0.75,1,0.75


In [24]:
df3 = pd.merge(df[['donor', 'receiver', 'compound', 'rate', 'frequency']].query(
    'donor != "environment" and receiver != "environment"'),
         df2[['donor', 'receiver', 'compound', 'smetana']],
         on=['donor', 'receiver', 'compound'])

In [27]:
df3.query('frequency > 0.1 and smetana > 0.1').sort_values('compound')

,donor,receiver,compound,rate,frequency,smetana
5,L_lactis,A_fabarum,M_4abut_e,0.038957,0.23,0.18
1,L_lactis,L_kefiranofaciens,M_acald_e,0.551990,0.62,0.29
4,L_mesenteroides,L_kefiranofaciens,M_acald_e,0.103917,0.44,0.29
7,L_lactis,L_mesenteroides,M_acald_e,0.035562,0.14,0.17
16,A_fabarum,L_kefiranofaciens,M_acald_e,0.005055,0.14,0.29
23,L_kefiranofaciens,A_fabarum,M_cys__L_e,0.000616,0.89,0.23
0,L_kefiranofaciens,L_lactis,M_gal_e,0.416310,0.84,0.22
2,L_lactis,L_mesenteroides,M_glyc_e,0.129423,0.72,0.34
11,L_lactis,A_fabarum,M_glyc_e,0.005391,1.00,0.18
14,L_mesenteroides,L_kefiranofaciens,M_h2s_e,0.006697,0.44,0.88
